In [5]:
#*****************************************************************************************************
#
# Checking the existence of a 5-chromatic unit distance graph embedded in the plane
#
#              Python 3 / Sage 9.2 / Numba code,
#
#              tested only under Ubuntu 18.04, 20.04
#
#
# We hope that this code still can be useful when the number of vertices is up to ~ 3*10^5
#
# Vsevolod Voronov, Anna Neopryatnaya, Eugene Dergachev, May 2021
#
#     email: v-vor(at)yandex.ru
#

!sage -pip install numba    # required for computing edge list of a distance graph in a reasonable time
!sage -pip install tqdm     # displaying progress bars -- just for convenience
from tqdm import tqdm
import numpy as np
import time
from numba import jit,njit
from math import pi
import networkx as nx


You should consider upgrading via the '/home/polaris/SageMath/SageMath/local/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/polaris/SageMath/SageMath/local/bin/python3 -m pip install --upgrade pip' command.


In [41]:
intmul = 10000000000    # for computing 'hash' strings 

eps = float(1E-10)      # used in the numerical verification of equalities u==v, |u-v|==1

def initial_set_moser():      # 31 vertices, including the origin
    a = (sqrt(33) + I*sqrt(3))/6
    r60 = 1/2 +I*sqrt(3)/2
    vect = [0+0*I]
    vect += [r60^i for i in range(6)]
    vect += [a*(r60^i) for i in range(6)]
    vect += [a**2*(r60^i) for i in range(6)]
    vect += [conjugate(a)*(r60^i) for i in range(6)]
    vect += [conjugate(a)**2*(r60^i) for i in range(6)]
    for i in range(len(vect)):
        vect[i] = vect[i].full_simplify().canonicalize_radical()
    vectn = [v.n() for v in vect]
    
    return vect, vectn


def initial_set_sqrt2():     # 73 vertices, including the origin
    a=(1/2)+sqrt(1/6) + I*(sqrt(1/2)-1/(2*sqrt(3)))
    r15=(sqrt(3)-1)/(2*sqrt(2))*I + (sqrt(3)+1)/(2*sqrt(2))
    vect = [0+0*I]
    vect += [r15^i for i in range(24)]
    vect += [a*(r15^i) for i in range(24)]
    vect += [conjugate(a)*(r15^i) for i in range(24)]
    for i in range(len(vect)):
        vect[i] = vect[i].full_simplify().canonicalize_radical()
    vectn = [v.n() for v in vect]
    
    return vect, vectn

def inthash(z):
    return int((z.real()+0.5)*intmul)+I*int((z.imag()+0.5)*intmul)


def strhashes(z):
    i1 = int((z.real()+0.5)*intmul)
    j1 = int((z.imag()+0.5)*intmul)
    return [str(i1)+"+i"+str(j1),str(i1-1)+"+i"+str(j1),str(i1+1)+"+i"+str(j1),   str(i1)+"+i"+str(j1-1),str(i1-1)+"+i"+str(j1-1),str(i1+1)+"+i"+str(j1-1),    str(i1)+"+i"+str(j1+1),str(i1-1)+"+i"+str(j1+1),str(i1+1)+"+i"+str(j1+1)] 


def exist(v,res,resn,hh,eps=eps):
    h = strhash(v)
    if h in hh:
        return True
    vn = v.n()
    for i in range(len(res)):
        un = resn[i]
        dn = un-vn
        if dn*conjugate(dn)<eps:
            d = res[i] - v
            if (d*conjugate(d)).simplify_full().canonicalize_radical() != 0:
                print("Warning: a distance is less than epsilon, but not equal to zero, d = ",d)
            else:
                return True
    return False        
                

def addnexist(v,cdict):
    hh = strhashes(v)
    for h in hh:
        if not cdict.get(h) is None:
            return True
    for h in hh:
        cdict[h] = v
    return False    


def mink_add(U,Un,V,Vn,clip = 2.0):
    res = []
    cdict = {}
    k = 0
    n = len(U)
    m = len(V)
    clip2 = clip*clip + eps
    for i in tqdm(range(n)):
        for j in range(m):
            s = Un[i]+Vn[j]
            if abs(s)>clip2:
                continue
            if not addnexist(s,cdict):
                res.append(U[i]+V[j])             
    resn = []
    for v in tqdm(res):
        resn.append(v.n())        
    return res,resn


one = float(1.0)   # to resolve some issues with Numba and Sage

@njit
def fast_select(D,n1,one,eps):
    return [(a,b) for a in range(n1) for b in range(a,n1) if ((a<b) and (abs(D[a,b]-one)<eps))]

@njit
def fast_select_sets(D,n1,n2,one,eps):
    return [(a,b) for a in range(n1) for b in range(n2) if (abs(D[a,b]-one)<eps)]


def get_edge_list(M, info = 0):      # consumes ~10 Gb RAM for 30k pts
    n1 = M.shape[0]

    start = time.time()
    M1 = M[:,:]
    SQ = np.sum(M1**2,1).reshape((1,n1))
    D = -2*M1.dot(M1.T )+ SQ + SQ.T

    edges = fast_select(D,n1,one,eps)
    finish = time.time()
    if info>0:
        print("{:.2f} s".format(finish-start))
        print("edges: ", len(edges))
    return(edges)

from itertools import product

def get_edge_list_blocks(X,blocksize=10000, info=0):    # consumes ~1 Gb RAM
    n = X.shape[0]
    bn = (n-1) // blocksize +1
    start = time.time()
    edges = []
    r = range(bn)
    quad = product(r,r)
    for q in tqdm(list(quad)):
        off1 = q[0]*blocksize 
        off2 = q[1]*blocksize
        A = X[off1:off1+blocksize,:]
        n1 = A.shape[0]
        B = X[off2:off2+blocksize,:]        
        n2 = B.shape[0]
        SQ1 = np.sum(A**2,1).reshape((1,n1))
        SQ2 = np.sum(B**2,1).reshape((1,n2))
        D = -2*A.dot(B.T )+ SQ1.T + SQ2
        ed = fast_select_sets(D,n1,n2,one,eps)
        for e in ed:
            u,v = e[0]+off1,e[1]+off2
            if u<v:
                edges.append((u,v))                
    finish = time.time()
    if info>0:
        print("{:.2f} s".format(finish-start))
        print("edges: ", len(edges))
    return(edges)

def get_edge_list_sets(X,Y, blocksize=10000, info=0):
    n = X.shape[0]
    m = Y.shape[0]
    bnx = (n-1) // blocksize +1
    bny = (m-1) // blocksize +1
    start = time.time()
    edges = []
    quad = product(range(bnx),range(bny))
    for q in tqdm(list(quad)):
        off1 = q[0]*blocksize 
        off2 = q[1]*blocksize
        A = X[off1:off1+blocksize,:]
        n1 = A.shape[0]
        B = Y[off2:off2+blocksize,:]        
        n2 = B.shape[0]
        SQ1 = np.sum(A**2,1).reshape((1,n1))
        SQ2 = np.sum(B**2,1).reshape((1,n2))
        D = -2*A.dot(B.T )+ SQ1.T + SQ2
        ed = fast_select_sets(D,n1,n2,one,eps)
        for e in ed:
            u,v = e[0]+off1,e[1]+off2
            #if u<v:
            edges.append((u,v))                
    finish = time.time()
    if info>0:
        print("{:.2f} s".format(finish-start))
        print("edges: ", len(edges))
    return(edges)

def list_to_matrix(arr):
    n = len(arr)
    M = np.zeros((n,2))
    for i in tqdm(range(n)):
        v = arr[i].n()
        M[i,0] = v.real()
        M[i,1] = v.imag()
    return M


def simp(e_len):    # for some reason the symbolic computation library can't do this simplification automatically
    s=str(e_len)
    s=s.replace(" ", "")
    s=s.replace("sqrt(-2*sqrt(2)+3)","(sqrt(2)-1)") 
    e_len=sage_eval(s,locals=globals())
    return e_len.full_simplify().canonicalize_radical()

def check_edges(sum3, L):
    cnt=0
    fails=[]
    for e in tqdm(L):
        if ((sum3[e[0]]-sum3[e[1]])*conjugate(sum3[e[0]]-sum3[e[1]])).full_simplify().canonicalize_radical() != 1:
            cnt+=1
            fails.append(e)
    return fails


def check_edges_symm(U, Un, L, n, info = 1):
    cnt_all = 0
    cnt_f = 0
    fails = []
    
    z = exp(2*pi*I/n)
    tg = z.imag()/z.real()+eps
    for e in tqdm(L):
        v = Un[e[0]]
        if (v.real()<=-eps) or (v.imag()<=-eps) or (v.imag()/v.real()>tg):
            continue
        cnt_all += 1    
        if ((U[e[0]]-U[e[1]])*conjugate(U[e[0]]-U[e[1]])).full_simplify().canonicalize_radical() != 1:
            cnt_f += 1
            fails.append(e)
    if info > 0:
        print("checked:", cnt_all)
        print("errors:", cnt_f)
    return fails


def check_new_edges(sum3, L, psi, info=1):
    cnt=0
    fails=[]
    for e in tqdm(L):
        e_len =((sum3[e[0]+1]-(sum3[e[1]+1]*psi))*conjugate(sum3[e[0]+1]-(sum3[e[1]+1]*psi))).full_simplify().canonicalize_radical()
        if e_len != 1:            
            e_len=simp(e_len)  
            if e_len !=1:
                e_len=simp(e_len)
                if e_len !=1:
                    cnt+=1
                    if info>0:
                        print("Not equal to 1: ", e_len)
                    fails.append(e)    
    return fails
    
def rotate_copy_n(U,psi):
    res = []
    psin = psi.n()

    for v in tqdm(U):
        res.append(v.n()*psin)
    return res


def points_to_rotation(p1,p2,phi0,symm_order=24): 
    phi01 = phi0.n()^(0.5)
    l1 = p1*conjugate(p1)
    l2 = p2*conjugate(p2)
    c = (1-l1-l2)/(-2*(l1*l2)**(1/2))
    c0 = -1/(p2/(p2*conjugate(p2))**(1/2))
    c1 = c + I*(1-c*c)**(1/2)
    ang = conjugate((c0/c1)).full_simplify()
    k = 0
    while not(-phi01.imag()<ang.n().imag()<phi01.imag()):
        ang*=phi0
        #ang = ang.full_simplify()
        k+=1
        if k>symm_order:
            break
    if ang.n().imag()<0:
        ang=conjugate(ang).full_simplify()
    return ang


def matrix_to_graph(M):     # M is (nx2) matrix
    L = get_edge_list_blocks(M)
    return nx.Graph(L)

def two_lists_to_graph(L,L1,n):
    La = [(u+n,v+n) for u,v in L if u>0]
    Lb = [(u,v+n) for u,v in L if u==0]
    Lc = [(u+1,v+n+1) for u,v in L1]
    return nx.Graph(L+La+Lb+Lc)


def write_dimacs_graph(g,filename):
    f = open(filename,"w")
    g = nx.convert_node_labels_to_integers(g,1)
    f.write("p edge {} {}\n".format(g.number_of_nodes(),g.number_of_edges()))
    for u, v in g.edges():
        f.write("e {} {}\n".format(u, v))
    f.close()

def int_list_str(lst):
        s = ""
        for p in lst:
            s = s + str(p)+" "
        s = s +"0\n"   
        return s      
    
def write_CNF(g, filename, ncolors = 4):        
    g = nx.convert_node_labels_to_integers(g,0)
    #g = nx.convert_node_labels_to_integers(g,0,ordering="decreasing degree")
    
    f = open(filename,"w")
        
    clique = next(nx.find_cliques(g))
    
    f.write("p cnf {} {}\n".format(g.number_of_nodes()*ncolors,g.number_of_nodes()+g.number_of_edges()*ncolors+len(clique)))
    
    col = 1
    for v in clique:
        f.write(int_list_str([v*ncolors+col]))        
        col += 1
    
    for u, v in g.edges():
        for c in range(1,ncolors+1):
            f.write("-{} -{} 0\n".format(u*ncolors+c, v*ncolors+c))
            
    for i in g.nodes():
        f.write(int_list_str(list(range(i*ncolors+1,i*ncolors+ncolors+1))))
                        
    f.close()

    
    
def psi_series1():    # returns the list of rotations for which udg(M_3+psi*M_3) if 5-chromatic, Series 1, 3877 vertices
    psi = 7/8 + I*sqrt(15)/8
    phi1 = (sqrt(33) + I*sqrt(3))/6
    phi0 = 1/2 +I*sqrt(3)/2
    return [psi, phi0*conjugate(phi1)*conjugate(psi),conjugate(phi1)*psi,phi1**2*conjugate(psi),conjugate(phi0)*phi1**2*psi]

    
def psi_series2():     # returns the list of rotations for which udg(M_3+psi*M_3) if 5-chromatic, Series 2, 64513 vertices 
    phi0=(sqrt(3)-1)/(2*sqrt(2))*I + (sqrt(3)+1)/(2*sqrt(2))
    phi1=(1/2)+sqrt(1/6) + I*(sqrt(1/2)-1/(2*sqrt(3)))

    key_points = [
        [2,1+phi0^5],
        [2,1+phi0],
        [2,1+phi0^7],
        [2,phi1*(1+phi0^7)],
        [2,phi1*(1+phi0^5)],
        [2,phi1*(1+phi0)],
        [2,conjugate(phi1)*(1+phi0)],
        [2,conjugate(phi1)*(1+phi0^5)],
        [2,conjugate(phi1)*(1+phi0^7)],
        [2,2],
        [2*(phi1*phi0^5).real(), 2*(phi1*phi0^2).real()]
    ]

    psi_arr = []
    k = 1
    for u,v in key_points:
        psi = points_to_rotation(u,v,phi0)
        psi_arr.append(psi)
        print("Case",k,": psi = ", psi.n())
        k += 1
    return psi_arr    


def check_dist_graph(psi, series=1, fname_dimacs = "", fname_cnf="", check_common_edges = False, s3=[], s3n=[]):
    if series == 1:
        print("Series 1: The basic element is the Moser spindle, \n the corresponding algebraic axtension is Q(sqrt(3),sqrt(11))")
    else:     
        print("Series 2: The basic element is the 10-vertex graph L_{10,1}, \n the corresponding algebraic axtension is Q(sqrt(2),sqrt(3))")
    print("psi = ", psi.n())    
    
    if not s3 or not s3n:
        print("Computing  vertex sets M_1, M_2, M_3...")
        if series == 1:
            vect, vectn = initial_set_moser()    
        else:     
            vect, vectn = initial_set_sqrt2()
        
        s2,s2n = mink_add(vect,vectn,vect,vectn,1.0)
        print(len(s2), "nodes in M_2 = clip(M_1 + M_1, 1.0)")
        s3,s3n = mink_add(s2,s2n,vect,vectn)

    print(len(s3), "nodes in M_3 = M_2 + M_1")

    print("Computing numerical values ...")
    
    s3na = rotate_copy_n(s3n,psi)
    P = list_to_matrix(s3n)
    Q = list_to_matrix(s3na)

    print("Computing edge list ...")

    L = get_edge_list_blocks(P)

    L1 = get_edge_list_sets(P[1:,:],Q[1:,:])    # exclude the origin

    print(len(L),"edges in udg(M_3)")

    print(len(L1),"new edges (connecting M_3 and psi*M_3)")

    print(2*len(L)+len(L1)," edges in the graph")    
    
    if check_common_edges:
        print("Checking common edges (up to symmetry)")
        check_edges_symm(s3,s3n,L,24)
        
    print("Checking new edges...")    
    err =  check_new_edges(s3,L1,psi)
    if err:
        print(len(err), "errors")
    else:
        print("All edge lengths are equal to 1")
    
    print("Converting to the NetworkX graph...")    
    g = two_lists_to_graph(L,L1,len(s3))
    
    if fname_dimacs:
        print("Writing a graph data (DIMACS) ...")    
        write_dimacs_graph(g,fname_dimacs)
        
    if fname_cnf:
        print("Writing the CNF (existence of 4-coloring) ...")    
        write_CNF(g,fname_cnf)
    
    print("Done!")
    
    return s3,s3n,s3na,g
    



In [44]:
# ****************************************************************************************
#
#  Checking the existence of the 5-chromatic graph from the first series (3877 vertices)
#
#  

psi_s1 = psi_series1()
for p in psi_s1:
    print(p.n())



0.875000000000000 + 0.484122918275927*I
0.969160753429815 + 0.246429369214292*I
0.977502967880533 + 0.210921662673018*I
0.996775678722190 + 0.0802386833635565*I
0.999038808433311 + 0.0438344527073038*I


In [45]:
case = 4    # 1 to 5

dimacs_name = 'graph{}_s1.dimacs'.format(case)
cnf_name = 'graph{}_s1.cnf'.format(case)
m3, m3n, psi_m3n, g = check_dist_graph(psi_s1[case-1], 1, dimacs_name, cnf_name)

Series 1: The basic element is the Moser spindle, Q(sqrt(3),sqrt(11))
psi =  0.996569973458116 + 0.0827543835798989*I
Computing  vertex sets M_1, M_2, M_3...


  0%|          | 0/1939 [00:00<?, ?it/s]

151 nodes in M_2 = clip(M_1 + M_1, 1.0)


  0%|          | 0/1 [00:00<?, ?it/s]

1939 nodes in M_3 = M_2 + M_1
Computing numerical values ...
Computing edge list ...


100%|██████████| 1/1 [00:00<00:00, 30.30it/s]
0it [00:00, ?it/s]


13374 edges in udg(M_3)
0 new edges (connecting M_3 and psi*M_3)
26748  edges in the graph
Checking new edges...
All edge lengths are equal to 1
Converting to the NetworkX graph...
Writing a graph data (DIMACS) ...
Writing the CNF (existence of 4-coloring) ...
Done!


In [ ]:
# # Uncomment and run the command below to install minisat (if needed)

#!sudo apt install minisat

In [34]:
# It is recommended to download Glucose 4.1 source https://www.labri.fr/perso/lsimon/glucose/
# make the executable and move it to the SageMath folder.
#
#  Glucose 4 is 3-5 times faster than Minisat when solving distance graph coloring problems,
#  and the multithreaded glucose-syrup is 2-3 times faster then the single-threaded glucose, 
#  running on 4 or more cores.
#
# ./glucose name.cnf
# ./glucose-syrup name.cnf
#  


!minisat graph4_s1.cnf


============================[ Problem Statistics ]=============================
|                                                                             |
|  Number of variables:         15508                                         |
|  Number of clauses:          107983                                         |
|  Parse time:                   0.02 s                                       |
|  Eliminated clauses:           0.10 Mb                                      |
|  Simplification time:          0.07 s                                       |
|                                                                             |
============================[ Search Statistics ]==============================
| Conflicts |          ORIGINAL         |          LEARNT          | Progress |
|           |    Vars  Clauses Literals |    Limit  Clauses Lit/Cl |          |
|       100 |   11508   103991   307524 |    38130      100     24 |  0.793 % |
|       250 |   11508   103991   307524 

In [43]:
# ****************************************************************************************
#
#  Checking the existence of the 5-chromatic graph from the second series (64513 vertices)
#
#  


psi_s2 = psi_series2()   # here symbolic computations takes a while

In [46]:
case = 10    # 1 to 11

dimacs_name = 'graph{}_s2.dimacs'.format(case)
cnf_name = 'graph{}_s2.cnf'.format(case)
m3, m3n, psi_m3n, g = check_dist_graph(psi_s2[case-1], 2, dimacs_name, cnf_name)


Series 2: The basic element is the 10-vertex graph L_{10,1}, Q(sqrt(2),sqrt(3))
psi =  0.999833687449347 + 0.0182372542187895*I
Computing  vertex sets M_1, M_2, M_3...


  3%|▎         | 26/865 [00:00<00:03, 257.88it/s]

865 nodes in M_2 = clip(M_1 + M_1, 1.0)


 13%|█▎        | 4127/32257 [00:00<00:03, 9257.57it/s]

32257 nodes in M_3 = M_2 + M_1
Computing numerical values ...


...

Computing edge list ...


  3%|▎         | 4/120 [00:00<00:04, 27.94it/s]

271176 edges in udg(M_3)
120 new edges (connecting M_3 and psi*M_3)
542472  edges in the graph
Checking new edges...


100%|██████████| 120/120 [00:05<00:00, 21.11it/s]


All edge lengths are equal to 1
Converting to the NetworkX graph...
Writing a graph data (DIMACS) ...
Writing the CNF (existence of 4-coloring) ...
Done!


In [49]:
!./glucose-syrup graph10_s2.cnf

c
c This is glucose-syrup 4.0 (glucose in many threads) --  based on MiniSAT (Many thanks to MiniSAT team)
c
c WARNING: for repeatability, setting FPU to use double precision
c ========================================[ Problem Statistics ]===========================================
c |                                                                                                       |
c |  Number of variables:        258052                                                                   |
c |  Number of clauses:         2217609                                                                   |
c |  Parse time:                   0.59 s                                                                 |
c |                                                                                                       |
c |  Eliminated clauses:           1.72 Mb                                                                |
c |  Simplification time:          3.28 s                            

c
c |-------------------------------------------------------------------------------------------------------|
c | id | starts | decisions  |  confls    |  Init T  |  learnts | exported | imported | promoted |    %   | 
c |-------------------------------------------------------------------------------------------------------|
c |  0 |    109 |      89927 |      65004 |   193336 |    26462 |     8766 |    30613 |      220 |  0.113 |
c |  1 |     64 |      96272 |      68228 |   204258 |    29695 |     9601 |    29659 |      220 |  0.113 |
c |  2 |    126 |      98296 |      69777 |   219038 |    16222 |    10047 |    30377 |      141 |  0.113 |
c |  3 |    140 |     107415 |      77468 |   206974 |    23907 |    12032 |    28230 |       29 |  0.113 |
c 
c synthesis      280477 conflicts   289819303 propagations     1896 conflicts/sec  1959009 propagations/sec
c Total Memory so far : 1206.75Mb
c
c |-------------------------------------------------------------------------------------------

c
c |-------------------------------------------------------------------------------------------------------|
c | id | starts | decisions  |  confls    |  Init T  |  learnts | exported | imported | promoted |    %   | 
c |-------------------------------------------------------------------------------------------------------|
c |  0 |    188 |     170159 |     130606 |   268964 |    39237 |    18168 |    52029 |     1353 |  0.113 |
c |  1 |    132 |     170876 |     127978 |   296645 |    36599 |    17307 |    49596 |      928 |  0.113 |
c |  2 |    269 |     169682 |     124256 |   296703 |    32885 |    16372 |    52444 |      472 |  0.113 |
c |  3 |    404 |     188410 |     140306 |   292200 |    26121 |    18405 |    51719 |      112 |  0.113 |
c 
c synthesis      523146 conflicts   556823858 propagations     1793 conflicts/sec  1908940 propagations/sec
c Total Memory so far : 1216.70Mb
c
c |-------------------------------------------------------------------------------------------

c
c |-------------------------------------------------------------------------------------------------------|
c | id | starts | decisions  |  confls    |  Init T  |  learnts | exported | imported | promoted |    %   | 
c |-------------------------------------------------------------------------------------------------------|
c |  0 |    259 |     262208 |     204860 |   378267 |    37295 |    25654 |    68271 |     3908 |  0.113 |
c |  1 |    277 |     255896 |     197485 |   348369 |    29907 |    24392 |    72369 |     2212 |  0.113 |
c |  2 |    459 |     249759 |     185754 |   352569 |    46168 |    22635 |    72076 |      723 |  0.113 |
c |  3 |    587 |     264564 |     200792 |   347132 |    33214 |    24108 |    72674 |     1292 |  0.113 |
c 
c synthesis      788891 conflicts   838397060 propagations     1747 conflicts/sec  1857139 propagations/sec
c Total Memory so far : 1195.37Mb
c
c |-------------------------------------------------------------------------------------------

c
c |-------------------------------------------------------------------------------------------------------|
c | id | starts | decisions  |  confls    |  Init T  |  learnts | exported | imported | promoted |    %   | 
c |-------------------------------------------------------------------------------------------------------|
c |  0 |    293 |     326728 |     260353 |   429859 |    62179 |    31342 |    85147 |     7873 |  0.113 |
c |  1 |    491 |     326816 |     253360 |   391499 |    55193 |    28998 |    88294 |     5806 |  0.113 |
c |  2 |    621 |     321873 |     241742 |   393553 |    43568 |    27895 |    87448 |      963 |  0.113 |
c |  3 |    709 |     334202 |     258919 |   373539 |    60745 |    29074 |    87007 |     3281 |  0.113 |
c 
c synthesis     1014375 conflicts  1096015318 propagations     1660 conflicts/sec  1793308 propagations/sec
c Total Memory so far : 1231.98Mb
c
c |-------------------------------------------------------------------------------------------

...

c
c |-------------------------------------------------------------------------------------------------------|
c | id | starts | decisions  |  confls    |  Init T  |  learnts | exported | imported | promoted |    %   | 
c |-------------------------------------------------------------------------------------------------------|
c |  0 |   1039 |     673592 |     530677 |   997235 |    94305 |    53999 |   170686 |    16690 |  0.113 |
c |  1 |   1225 |     703573 |     555807 |   805444 |    70347 |    54213 |   169446 |    16900 |  0.113 |
c |  2 |   1001 |     792314 |     631614 |   747763 |    95035 |    61694 |   154279 |    19645 |  0.113 |
c |  3 |   1527 |     724948 |     570913 |   755541 |    57570 |    54903 |   169789 |    11342 |  0.113 |
c 
c synthesis     2289011 conflicts  3217608706 propagations     1211 conflicts/sec  1702754 propagations/sec
c Total Memory so far : 1266.29Mb
c
c |-------------------------------------------------------------------------------------------

c
c |-------------------------------------------------------------------------------------------------------|
c | id | starts | decisions  |  confls    |  Init T  |  learnts | exported | imported | promoted |    %   | 
c |-------------------------------------------------------------------------------------------------------|
c |  0 |   1086 |     718407 |     567615 |  1003994 |    64927 |    57260 |   176785 |    16952 |  0.114 |
c |  1 |   1304 |     748847 |     592011 |   817816 |    38230 |    56946 |   176709 |    18106 |  0.114 |
c |  2 |   1019 |     831760 |     664804 |   758193 |    74113 |    64537 |   169649 |    20285 |  0.114 |
c |  3 |   1570 |     767301 |     605057 |   768349 |    54359 |    56601 |   178624 |    13373 |  0.114 |
c 
c synthesis     2429487 conflicts  3465414185 propagations     1185 conflicts/sec  1690841 propagations/sec
c Total Memory so far : 1268.35Mb
c
c |-------------------------------------------------------------------------------------------

c
c |-------------------------------------------------------------------------------------------------------|
c | id | starts | decisions  |  confls    |  Init T  |  learnts | exported | imported | promoted |    %   | 
c |-------------------------------------------------------------------------------------------------------|
c |  0 |   1216 |     768577 |     607827 |  1009852 |    62484 |    59482 |   182559 |    17132 |  0.114 |
c |  1 |   1395 |     797656 |     631737 |   829596 |    77956 |    57781 |   184001 |    19857 |  0.114 |
c |  2 |   1104 |     877209 |     702230 |   766780 |   111539 |    67238 |   174792 |    21129 |  0.114 |
c |  3 |   1676 |     819487 |     647902 |   775997 |    50259 |    57543 |   184472 |    15385 |  0.114 |
c 
c synthesis     2589696 conflicts  3707376825 propagations     1172 conflicts/sec  1677977 propagations/sec
c Total Memory so far : 1271.93Mb
c
c |-------------------------------------------------------------------------------------------

c
c |-------------------------------------------------------------------------------------------------------|
c | id | starts | decisions  |  confls    |  Init T  |  learnts | exported | imported | promoted |    %   | 
c |-------------------------------------------------------------------------------------------------------|
c |  0 |   1306 |     819479 |     649111 |  1023021 |    54147 |    61195 |   187802 |    17571 |  0.114 |
c |  1 |   1468 |     851583 |     677043 |   834619 |    77853 |    61595 |   192248 |    21645 |  0.114 |
c |  2 |   1161 |     926501 |     744060 |   774466 |    96821 |    69609 |   184253 |    22404 |  0.114 |
c |  3 |   1721 |     872802 |     693388 |   779191 |    95745 |    61590 |   188838 |    17146 |  0.114 |
c 
c synthesis     2763602 conflicts  3941833261 propagations     1166 conflicts/sec  1663653 propagations/sec
c Total Memory so far : 1275.24Mb
c
c |-------------------------------------------------------------------------------------------

c Thread 1 is 100% pure glucose! First thread to finish! (UNSAT answer).
c Total Memory so far : 1278.60Mb
c
c real time : 633.276 s
c cpu time  : 2515.7 s
c
c
c
c |---------------------------------------- FINAL STATS --------------------------------------------------|
c
c |---------------|-----------------|------------|------------|------------|------------|
c | Threads       |      Total      |          0 |          1 |          2 |          3 |
c |---------------|-----------------|------------|------------|------------|------------|
c | Conflicts     |         2915390 |     683434 |     715659 |     782455 |     733842 |
c | Decisions     |         3647927 |     861044 |     895669 |     970991 |     920223 |
c | Propagations  |      4155920906 | 1061774131 |  999417809 | 1036790176 | 1057938790 |
c | Avg_Trail     |                 |      40632 |      36216 |      39377 |      43190 |
c | Avg_DL        |                 |         25 |         27 |         28 |         27 |
c | Avg_